In [ ]:
import requests
import pandas as pd
import matplotlib
import numpy as np
from config import DATA_HOLIDAYS_PROCESSED_FILE

In [ ]:
API_KEY = '##b13b8edeebf4b7dd082e7df702fc94702bef7327##'
END_POINT='https://calendarific.com/api/v2/holidays?&api_key='
YEARS = ['2018','2019']
COUNTRY = 'SG'


In [ ]:
def f(x, hl):
    if x in hl:
        return 1
    else:
        return 0
def f_remove(x, hl):
    if x in hl:
        return 0
    else:
        return x[1]

In [ ]:
#get holidays from API
holidays_list = []
for year in YEARS:
    url = END_POINT+API_KEY+'&country='+COUNTRY+"&year="+year
    holidays_list_dicts = requests.get(url).json()['response']['holidays']
    holidays_list.extend([x['date']['iso'] for x in holidays_list_dicts])

In [ ]:
#clean it up
holidays_list = [x.split('T')[0] for x in holidays_list]
earliest_date = '2018-01-01'
latest_date = '2019-12-31'
holidays_list_datetime = pd.to_datetime(holidays_list)
data = pd.DataFrame({'timestamp':pd.date_range(start=earliest_date, end=latest_date, freq='D')})
data['holiday'] = data['timestamp'].apply(lambda x: f(x, holidays_list_datetime))

In [ ]:
school_holiday = pd.to_datetime(['2018-03-05',
               '2018-03-06',
              '2018-03-07',
               '2018-03-08',
               '2018-03-09',
               '2018-10-01',
               '2018-10-02',
               '2018-10-03',
               '2018-10-04',
               '2018-10-05',
               '2019-03-04',
              '2019-03-05',
              '2019-03-06',
               '2019-03-07',
               '2019-03-08',
               '2019-09-30',
               '2019-10-01',
              '2019-10-02',
               '2019-10-03',
               '2019-10-04'])
data['school_holiday'] = data['timestamp'].apply(lambda x: f(x, school_holiday))

In [ ]:
#get teaching dates form NTU calendar
teaching0 = pd.date_range(start='2018-01-15', end='2018-03-02', freq='D')
teaching1 = pd.date_range(start='2018-03-12', end='2018-05-11', freq='D')

teaching2 = pd.date_range(start='2018-08-13', end='2018-09-28', freq='D')
teaching3 = pd.date_range(start='2018-10-08', end='2018-12-07', freq='D')

teaching4 = pd.date_range(start='2019-01-14', end='2019-03-01', freq='D')
teaching5 = pd.date_range(start='2019-03-11', end='2019-05-10', freq='D')
teaching6 = pd.date_range(start='2019-08-13', end='2019-11-15', freq='D')

teaching_time_datetime = teaching0.append(teaching1).append(teaching2).append(teaching3).append(teaching4).append(teaching5).append(teaching6)
data['teaching_time'] =  data['timestamp'].apply(lambda x: f(x,teaching_time_datetime))

# take out public holidays and weekends (no teaching) 
data['dayofweek'] = np.array(data['timestamp'].dt.dayofweek, dtype=np.uint8)
data['weekend_days'] = data['dayofweek'].apply(lambda x: 1 if x>=5 else 0)
# data['teaching_time'] = data[['timestamp','teaching_time']].apply(lambda x: f_remove(x, holidays_list_datetime), axis=1)
# data['teaching_time'] = data[['teaching_time','weekend_days']].apply(lambda x: 0 if x[1]==1 else x[0], axis=1)

In [ ]:
visual = data.set_index('timestamp')
idx = ["","False","","","","","True"]
ax = visual[['holiday', 'school_holiday', 'teaching_time']].plot(color = ['C0', 'C9', 'C8'])
font = {'family' : 'Arial',
        'size'   : 18}
ax.set_xlabel("")
ax.set_ylabel("Boolean [-]")
ax.legend(["Public Holiday", 'School Holiday', "Working Day"]);
ax.set_yticklabels(idx)
matplotlib.rc('font', **font)

In [ ]:
visual = data.set_index('timestamp')
idx = ["","False","","","","","True"]
ax = visual[['teaching_time']].plot(color = ['C0', 'C9', 'C8'])
font = {'family' : 'Arial',
        'size'   : 18}
ax.set_xlabel("")
ax.set_ylabel("Boolean [-]")
ax.legend(["Public Holiday", 'School Holiday', "Working Day"]);
ax.set_yticklabels(idx)
matplotlib.rc('font', **font)

In [ ]:
data[['timestamp','holiday', 'school_holiday', 'teaching_time']].to_csv(DATA_HOLIDAYS_PROCESSED_FILE, index=False)